**420-A58-SF - Algorithmes d'apprentissage non supervisé - Hiver 2023 - Spécialisation technique en Intelligence Artificielle**<br/>
MIT License - Copyright (c) 2023 Mikaël Swawola
<br/>
![Travaux Pratiques - Recherche de documents](static/02-02-A1-banner.png)
<br/>
**Objectif: Lors de l'exploration d'un jeu de données constitué de documents textes - tels que des pages Wikipedia, des articles de presse, StackOverflow, etc., il est courant de chercher à trouver quels sont les documents similaires. L'objectif de cet exercice est de mettre en oeuvre les techniques de recherche adaptées (ici les plus proches voisins) à ce type de données. Les documents utilisés sont les pages Wikipedia de personnalités.**

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

# Le reste des modules sera importé au fur et à mesure des exercices ...

L'archive `people.zip` contient 4 fichiers:

* **people_wiki.csv**: jeu de données consituté des pages Wikipedia de personnalités
* **people_wiki_map_index_to_word.json**: mapping entre les mots et les indices
* **people_wiki_word_count.npz**: vecteurs d'occurence des mots (word count / sacs de mot) pour chaque document
* **people_wiki_tf_idf.npz**: vecteurs TF-IDF pour chaque document

Dans l'énoncé de ce TP, les mots "article" et "document" sont interchangeables.

## 1 - Chargement du jeu de données

**Exercice 1-1 - À l'aide de la librairie Pandas, lire le fichier de données `people/people_wiki.csv`. Afin de permettre les opérations de type `join` effectuées plus loin dans le TP, nommez l'index de la trame de donnée `id`**

In [3]:
# Compléter cette cellule ~ 2 lignes de code
wiki = pd.read_csv("../../../Data/people_wiki.csv")

**Exercice 1-2 - Afficher les 5 premières lignes de la trame de données. Quelles informations contiennent les colonnes ?**

In [4]:
# Compléter cette cellule ~ 1 ligne de code
wiki.head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...


## 2 - Extraction du nombre de mots

Les vecteurs d'occurence des mots (**word count**) du jeu de données ont été préalablement extrait dans le fichier `people/people_wiki_word_count.npz`. Ces vecteurs sont regroupés dans une matrice diluée (sparse), où la i-ème ligne donne le vecteur d'occurence des mots pour le i-ème document. Chaque colonne correspond à un mot unique apparaissant dans le jeu de données. Le mapping entre les mots et les indices est donné dans `people/people_wiki_map_index_to_word.json`

La fonction suivante permet le chargement des vecteurs d'occurence des mots:

In [5]:
from scipy.sparse import csr_matrix

def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

La fonction ci-dessus utilise `csr_matrix` de la bibliothèque SciPy:<br/>
[class scipy.sparse.csr_matrix(arg1, shape=None, dtype=None, copy=False)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)

**Exercice 2-1 - À l'aide de la fonction ci-dessus, charger le ficher contenant les vecteurs d'occurence des mots**

In [6]:
# Compléter cette cellule ~ 1 ligne de code
matrix = load_sparse_csr("../../../Data/people_wiki_word_count.npz")

**Exercice 2-2 - En vous référant à la documentation de la fonction `csr_matrix`, convertissez la matrice `word_count` en tableau NumPy. Que constatez-vous ?**

In [8]:
# Compléter cette cellule ~ 1 ligne de code
tableau = matrix.toarray(order='C')

MemoryError: Unable to allocate 241. GiB for an array with shape (59071, 547979) and data type int64

**Exercice 2-3 - À l'aide du module json ou de la librairie Pandas, charger le ficher contenant le mapping entre les mots et les indices. Combien y a-t-il de mots dans le dictionnaire ?**

In [14]:
# Compléter cette cellule ~ 2-3 lignes de code
mapJson = pd.read_json("../../../Data/people_wiki_map_index_to_word.json", typ='series')
len(mapJson)

547979

**Exercice 2-4 (optionnel) - Extraire par vous-même les vecteurs d'occurence des mots. Un bon point de départ est la fonction `sklearn.CountVectorizer`**

In [23]:
# Compléter cette cellule
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
word_count = vectorizer.fit_transform(wiki['text'])


## 3 - Recherche des plus proches voisins avec représentation word count

Commençons par trouver les voisins les plus proches de la page Wikipedia de **Barack Obama**. Les vecteurs d'occurence des mots (**word count**) seront utilisés pour représenter les articles et la **distance euclidienne** pour mesurer la similarité.

[class sklearn.neighbors.NearestNeighbors(*, n_neighbors=5, radius=1.0, algorithm='auto', leaf_size=30, metric='minkowski', p=2, metric_params=None, n_jobs=None)](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)

**Exercice 3-1 - Quel est l'id correspondant à la page Wikipedia de barack Obama ?**

In [11]:
from sklearn.neighbors import NearestNeighbors

In [13]:
# Compléter cette cellule ~ 1 ligne de code
wiki[wiki['name'].str.contains('obama', case=False)]

,URI,name,text
2936,<http://dbpedia.org/resource/Ricardo_Mangue_Ob...,Ricardo Mangue Obama Nfubea,ricardo mangue obama nfubea born c 1961 is a p...
35817,<http://dbpedia.org/resource/Barack_Obama>,Barack Obama,barack hussein obama ii brk husen bm born augu...


**Exercice 3-2 - À l'aide de scikit-learn, rechercher les 10 pages Wikipedia de personnalités les plus similaires à la page de Barack Obama. Affichez les distances et noms de personalités dans une même trame de données**

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances

In [28]:
### Gabarito
model = NearestNeighbors().fit(word_count)
model.kneighbors(word_count[35817], n_neighbors=10)
distances, indices = model.kneighbors(word_count[35817], n_neighbors=10+1)
neighbors = pd.DataFrame({'distances':distances.flatten(), 'id': indices.flatten()}).set_index('id')
neighbors.head()


,distances
id,
35817,0.000000
24478,33.015148
28447,34.307434
14754,35.791060
35357,36.069378


In [38]:

wiki.head()
print(wiki.index.name)
wiki.index.name='id'
wiki.join(neighbors, on='id', how='right')

id


,URI,name,text,distances
id,,,,
35817,<http://dbpedia.org/resource/Barack_Obama>,Barack Obama,barack hussein obama ii brk husen bm born augu...,0.000000
24478,<http://dbpedia.org/resource/Joe_Biden>,Joe Biden,joseph robinette joe biden jr dosf rbnt badn b...,33.015148
28447,<http://dbpedia.org/resource/George_W._Bush>,George W. Bush,george walker bush born july 6 1946 is an amer...,34.307434
14754,<http://dbpedia.org/resource/Mitt_Romney>,Mitt Romney,willard mitt romney born march 12 1947 is an a...,35.791060
35357,<http://dbpedia.org/resource/Lawrence_Summers>,Lawrence Summers,lawrence henry larry summers born november 30 ...,36.069378
31423,<http://dbpedia.org/resource/Walter_Mondale>,Walter Mondale,walter frederick fritz mondale born january 5 ...,36.249138
13229,<http://dbpedia.org/resource/Francisco_Barrio>,Francisco Barrio,francisco javier barrio terrazas born november...,36.276714
36364,<http://dbpedia.org/resource/Don_Bonker>,Don Bonker,don leroy bonker born march 7 1937 in denver c...,36.400549
22745,<http://dbpedia.org/resource/Wynn_Normington_H...,Wynn Normington Hugh-Jones,sir wynn normington hughjones kb sometimes kno...,36.441734


In [ ]:
###############

In [18]:
# Compléter cette cellule ~ 5-6 lignes de code

df = wiki
# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Transform the text data into a matrix of TF-IDF features
tfidf_matrix = vectorizer.fit_transform(df['text'])

# Compute pairwise distances between all rows
dist_matrix = pairwise_distances(tfidf_matrix, metric='cosine')

# Convert the distance matrix into a pandas DataFrame
dist_df = pd.DataFrame(dist_matrix, columns=df.index, index=df.index)

# Print the resulting DataFrame
print(dist_df)



MemoryError: Unable to allocate 26.0 GiB for an array with shape (3489324583,) and data type int64

**Exercice 3-3 - Interprétez les résultats ci-dessus**

In [ ]:
# Compléter cette cellule

**Exercice 3-4 - Affichez les mots les plus fréquents des pages de Barack Obama et Francisco Barrio**

Afin de pouvoir reconnaître rapidement les mots d'une grande importance, la fonction suivante permettant d'obtenir la colonne `word_count` est fournie.

In [39]:
def unpack_dict(matrix, map_index_to_word):
    table = sorted(map_index_to_word, key=map_index_to_word.get)
   
    data = matrix.data
    indices = matrix.indices
    indptr = matrix.indptr
    
    num_doc = matrix.shape[0]

    return [{k:v for k,v in zip([table[word_id] for word_id in indices[indptr[i]:indptr[i+1]] ],
                                 data[indptr[i]:indptr[i+1]].tolist())} for i in range(num_doc) ]

In [50]:
# Compléter cette cellule ~ 2 lignes de code
wiki['word_count'] = unpack = unpack_dict(word_count, map_index_to_word)

NameError: name 'map_index_to_word' is not defined

In [ ]:
################

In [46]:
df2 = wiki[wiki['name'].str.contains('Barack Obama') | wiki['name'].str.contains('Francisco Barrio')]
df2

,URI,name,text
id,,,
13229,<http://dbpedia.org/resource/Francisco_Barrio>,Francisco Barrio,francisco javier barrio terrazas born november...
35817,<http://dbpedia.org/resource/Barack_Obama>,Barack Obama,barack hussein obama ii brk husen bm born augu...


In [51]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

# Create a sample dataframe with a 'text' column
#df = pd.DataFrame({'text': ['this is a sample text', 'this is another sample text', 'yet another text']})
#df = wiki['text']

# Create a CountVectorizer object with English stopwords
stop_words = stopwords.words('english')
vectorizer = CountVectorizer(stop_words=stop_words)

# Transform the 'text' column into a matrix of word counts
word_counts = vectorizer.fit_transform(df2['text'])

# Iterate over the rows
for index, row in df2.iterrows():
    # Transform the row into a matrix of word counts
    word_counts = vectorizer.fit_transform([row['text']])
    
    # Get the vocabulary of words
    vocabulary = vectorizer.get_feature_names()

    # Sum the word counts across the row
    word_count_totals = word_counts.sum(axis=0)

    # Convert the word count totals to a pandas Series
    word_count_series = pd.Series(word_count_totals.A1, index=vocabulary)

    # Get the top 10 most frequent words
    top_10_words = word_count_series.nlargest(10)

    # Print the top 10 words for the row
    print(f"Top 10 words for row {index}:")
    print(top_10_words)

Top 10 words for row 13229:
chihuahua    7
governor     6
state        4
action       3
first        3
former       3
national     3
party        3
years        3
barrio       2
dtype: int64
Top 10 words for row 35817:
obama         9
act           8
law           6
us            6
control       4
democratic    4
iraq          4
military      4
president     4
2004          3
dtype: int64


/home/hadoop/.local/lib/python3.11/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/hadoop/.local/lib/python3.11/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


**Exercice 3-5 - Créer une fonction `top_words`, permattant d'afficher les mots les plus fréquents d'une page donnée**

In [ ]:
# Compléter cette cellule ~ 10 lignes de code

## 4 - Recherche des plus proches voisins avec représentation TF-IDF

**Exercice 4 - Répétez les étapes des exercices de la partie 3 en utilisant cette fois-ci la représentation TF-IDF. Comparez avec les résultats obtenu par la représentation word count**

In [53]:
# Compléter cette cellule ~ 14-20 lignes de code
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Create a sample dataframe with a 'text' column
#df = pd.DataFrame({'text': ['this is a sample text', 'this is another sample text', 'yet another text']})
df = wiki

# Create a CountVectorizer object with English stopwords
stop_words = stopwords.words('english')
vectorizer = CountVectorizer(stop_words=stop_words)

# Fit the vectorizer on the 'text' column
tfidf_matrix = vectorizer.fit_transform(df['text'])

# Get the feature names (i.e., the vocabulary of words)
feature_names = vectorizer.get_feature_names()

# Sum the tf-idf scores across each document (i.e., row)
sum_scores = tfidf_matrix.sum(axis=0)

# Convert the sums to a pandas Series
sum_scores_series = pd.Series(sum_scores.A1, index=feature_names)

# Get the top 10 most frequent words
top_10_words = sum_scores_series.nlargest(10)

# Print the top 10 words
print(top_10_words)


/home/hadoop/.local/lib/python3.11/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


also          67731
university    59644
born          54633
first         50279
new           49604
one           35127
music         29888
national      29758
american      28804
years         28802
dtype: int64


### Fin de l'atelier 02-02-A1